# Create a RAG system using OpenVINO GenAI and LangChain

**Retrieval-augmented generation (RAG)** is a technique for augmenting LLM knowledge with additional, often private or real-time, data. LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs. The process of bringing the appropriate information and inserting it into the model prompt is known as Retrieval Augmented Generation (RAG).

[LangChain](https://python.langchain.com/docs/get_started/introduction) is a framework for developing applications powered by language models. It has a number of components specifically designed to help build RAG applications. In this tutorial, we’ll build a simple question-answering application over a text data source.

The tutorial consists of the following steps:

- Install prerequisites
- Download and convert the model from a public source using the [OpenVINO integration with Hugging Face Optimum](https://huggingface.co/blog/openvino).
- Compress model weights to 4-bit or 8-bit data types using [NNCF](https://github.com/openvinotoolkit/nncf)
- Create a RAG chain pipeline with [OpenVINO GenAI](https://github.com/openvinotoolkit/openvino.genai)
- Run Q&A pipeline

In this example, the customized RAG pipeline consists of following components in order, where embedding, rerank and LLM will be deployed with [OpenVINO GenAI](https://github.com/openvinotoolkit/openvino.genai) to optimize their inference performance.

![RAG](https://github.com/openvinotoolkit/openvino_notebooks/assets/91237924/0076f6c7-75e4-4c2e-9015-87b355e5ca28)

#### Table of contents:

- [Prerequisites](#Prerequisites)
- [Select model for inference](#Select-model-for-inference)
- [login to huggingfacehub to get access to pretrained model](#login-to-huggingfacehub-to-get-access-to-pretrained-model)
- [Convert model and compress model weights](#convert-model-and-compress-model-weights)
  - [LLM conversion and Weights Compression using Optimum-CLI](#LLM-conversion-and-Weights-Compression-using-Optimum-CLI)
    - [Weight compression with AWQ](#Weight-compression-with-AWQ)
  - [Convert embedding model using Optimum-CLI](#Convert-embedding-model-using-Optimum-CLI)
  - [Convert rerank model using Optimum-CLI](#Convert-rerank-model-using-Optimum-CLI)
- [Select device for inference and model variant](#Select-device-for-inference-and-model-variant)
  - [Select device for embedding model inference](#Select-device-for-embedding-model-inference)
  - [Select device for rerank model inference](#Select-device-for-rerank-model-inference)
  - [Select device for LLM model inference](#Select-device-for-LLM-model-inference)
- [Load model](#Load-model)
  - [Load embedding model](#Load-embedding-model)
  - [Load rerank model](#Load-rerank-model)
  - [Load LLM model](#Load-LLM-model)
- [Run QA over Document](#Run-QA-over-Document)

### Installation Instructions

This is a self-contained example that relies solely on its own code.

We recommend  running the notebook in a virtual environment. You only need a Jupyter server to start.
For details, please refer to [Installation Guide](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/README.md#-installation-guide).

<img referrerpolicy="no-referrer-when-downgrade" src="https://static.scarf.sh/a.png?x-pxid=5b5a4db0-7875-4bfb-bdbd-01698b5b1a77&file=notebooks/llm-rag-langchain/llm-rag-langchain.ipynb" />


<img referrerpolicy="no-referrer-when-downgrade" src="https://static.scarf.sh/a.png?x-pxid=5b5a4db0-7875-4bfb-bdbd-01698b5b1a77&file=notebooks/llm-rag-langchain/llm-rag-langchain-genai.ipynb" />


## Prerequisites

[back to top ⬆️](#Table-of-contents:)

Install required dependencies


In [1]:
import os
import requests
from pathlib import Path

if not Path("notebook_utils.py").exists():
    r = requests.get(
        url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py",
    )
    with open("notebook_utils.py", "w") as f:
        f.write(r.text)

if not Path("pip_helper.py").exists():
    r = requests.get(
        url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/pip_helper.py",
    )
    open("pip_helper.py", "w").write(r.text)

if not Path("cmd_helper.py").exists():
    r = requests.get(
        url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/cmd_helper.py",
    )
    with open("cmd_helper.py", "w") as f:
        f.write(r.text)


if not Path("genai_helper.py").exists():
    r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/genai_helper.py")
    open("genai_helper.py", "w").write(r.text)

from pip_helper import pip_install

os.environ["GIT_CLONE_PROTECTION_ACTIVE"] = "false"

pip_install("--pre", "-U", "openvino>=2024.2.0", "--extra-index-url", "https://storage.openvinotoolkit.org/simple/wheels/nightly")
pip_install("--pre", "-U", "openvino-tokenizers[transformers]", "--extra-index-url", "https://storage.openvinotoolkit.org/simple/wheels/nightly")
pip_install("--pre", "-U", "openvino-genai", "--extra-index-url", "https://storage.openvinotoolkit.org/simple/wheels/nightly")
pip_install(
    "-q",
    "--extra-index-url",
    "https://download.pytorch.org/whl/cpu",
    "git+https://github.com/huggingface/optimum-intel.git",
    "git+https://github.com/openvinotoolkit/nncf.git",
    "datasets",
    "accelerate",
    "gradio>=4.19",
    "onnx<1.16.2",
    "einops",
    "transformers_stream_generator",
    "tiktoken",
    "transformers>=4.43.1",
    "faiss-cpu",
    "sentence_transformers",
    "langchain>=0.2.0",
    "langchain-community>=0.2.15",
    "langchainhub",
    "unstructured",
    "scikit-learn",
    "python-docx",
    "pypdf",
    "huggingface-hub>=0.26.5",
)

# Read more about telemetry collection at https://github.com/openvinotoolkit/openvino_notebooks?tab=readme-ov-file#-telemetry
from notebook_utils import collect_telemetry

collect_telemetry("llm-rag-langchain-genai.ipynb")

Looking in indexes: https://pypi.org/simple, https://storage.openvinotoolkit.org/simple/wheels/nightly
Looking in indexes: https://pypi.org/simple, https://storage.openvinotoolkit.org/simple/wheels/nightly
Looking in indexes: https://pypi.org/simple, https://storage.openvinotoolkit.org/simple/wheels/nightly


In [2]:
import os
from pathlib import Path
import requests
import shutil
import io

# fetch model configuration

config_shared_path = Path("../../utils/llm_config.py")
config_dst_path = Path("llm_config.py")
text_example_en_path = Path("text_example_en.pdf")
text_example_cn_path = Path("text_example_cn.pdf")
text_example_en = "https://github.com/openvinotoolkit/openvino_notebooks/files/15039728/Platform.Brief_Intel.vPro.with.Intel.Core.Ultra_Final.pdf"
text_example_cn = "https://github.com/openvinotoolkit/openvino_notebooks/files/15039713/Platform.Brief_Intel.vPro.with.Intel.Core.Ultra_Final_CH.pdf"

if not config_dst_path.exists():
    if config_shared_path.exists():
        try:
            os.symlink(config_shared_path, config_dst_path)
        except Exception:
            shutil.copy(config_shared_path, config_dst_path)
    else:
        r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/llm_config.py")
        with open("llm_config.py", "w", encoding="utf-8") as f:
            f.write(r.text)
elif not os.path.islink(config_dst_path):
    print("LLM config will be updated")
    if config_shared_path.exists():
        shutil.copy(config_shared_path, config_dst_path)
    else:
        r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/llm_config.py")
        with open("llm_config.py", "w", encoding="utf-8") as f:
            f.write(r.text)

if not text_example_en_path.exists():
    r = requests.get(url=text_example_en)
    content = io.BytesIO(r.content)
    with open("text_example_en.pdf", "wb") as f:
        f.write(content.read())

if not text_example_cn_path.exists():
    r = requests.get(url=text_example_cn)
    content = io.BytesIO(r.content)
    with open("text_example_cn.pdf", "wb") as f:
        f.write(content.read())

## Select model for inference

[back to top ⬆️](#Table-of-contents:)

The tutorial supports different models, you can select one from the provided options to compare the quality of open source LLM solutions.

> **Note**: conversion of some models can require additional actions from user side and at least 64GB RAM for conversion.

The available embedding model options are:

- [**bge-small-en-v1.5**](https://huggingface.co/BAAI/bge-small-en-v1.5)
- [**bge-small-zh-v1.5**](https://huggingface.co/BAAI/bge-small-zh-v1.5)
- [**bge-large-en-v1.5**](https://huggingface.co/BAAI/bge-large-en-v1.5)
- [**bge-large-zh-v1.5**](https://huggingface.co/BAAI/bge-large-zh-v1.5)
- [**bge-m3**](https://huggingface.co/BAAI/bge-m3)

BGE embedding is a general Embedding Model. The model is pre-trained using RetroMAE and trained on large-scale pair data using contrastive learning.

The available rerank model options are:

- [**bge-reranker-v2-m3**](https://huggingface.co/BAAI/bge-reranker-v2-m3)
- [**bge-reranker-large**](https://huggingface.co/BAAI/bge-reranker-large)
- [**bge-reranker-base**](https://huggingface.co/BAAI/bge-reranker-base)

Reranker model with cross-encoder will perform full-attention over the input pair, which is more accurate than embedding model (i.e., bi-encoder) but more time-consuming than embedding model. Therefore, it can be used to re-rank the top-k documents returned by embedding model.

You can also find available LLM model options in [llm-chatbot](../llm-chatbot/README.md) notebook.

### Optional: Telegram Integration

This notebook also supports ingesting data from Telegram channels. To use this feature:

1. Get your Telegram API credentials:
   - Go to https://my.telegram.org/apps
   - Create a new application
   - Note down your `api_id` and `api_hash`

2. Create a `.env` file in this directory with your credentials:
   ```
   TELEGRAM_API_ID=your_api_id
   TELEGRAM_API_HASH=your_api_hash
   ```

3. Install additional dependencies:
   ```bash
   pip install -r requirements_telegram.txt
   ```

The Telegram integration allows you to:
- Download messages from specified Telegram channels
- Process them using OpenVINO-optimized models
- Add them to your RAG system's knowledge base
- Query across both document and Telegram sources


In [3]:
from pathlib import Path
import ipywidgets as widgets

## Convert model and compress model weights

[back to top ⬆️](#Table-of-contents:)

The Weights Compression algorithm is aimed at compressing the weights of the models and can be used to optimize the model footprint and performance of large models where the size of weights is relatively larger than the size of activations, for example, Large Language Models (LLM). Compared to INT8 compression, INT4 compression improves performance even more, but introduces a minor drop in prediction quality.

In [4]:
from llm_config import (
    SUPPORTED_EMBEDDING_MODELS,
    SUPPORTED_RERANK_MODELS,
    SUPPORTED_LLM_MODELS,
)

model_languages = list(SUPPORTED_LLM_MODELS)

model_language = widgets.Dropdown(
    options=model_languages,
    value=model_languages[0],
    description="Model Language:",
    disabled=False,
)

model_language

Dropdown(description='Model Language:', options=('English', 'Chinese', 'Japanese'), value='English')

In [5]:
llm_model_ids = [model_id for model_id, model_config in SUPPORTED_LLM_MODELS[model_language.value].items() if model_config.get("rag_prompt_template")]

llm_model_id = widgets.Dropdown(
    options=llm_model_ids,
    value=llm_model_ids[-1],
    description="Model:",
    disabled=False,
)

llm_model_id

Dropdown(description='Model:', index=19, options=('tiny-llama-1b-chat', 'llama-3.2-1b-instruct', 'llama-3.2-3b…

In [7]:
llm_model_configuration = SUPPORTED_LLM_MODELS[model_language.value][llm_model_id.value]
print(f"Selected LLM model {llm_model_id.value}")

Selected LLM model qwen2.5-3b-instruct


🤗 [Optimum Intel](https://huggingface.co/docs/optimum/intel/index) is the interface between the 🤗 [Transformers](https://huggingface.co/docs/transformers/index) and [Diffusers](https://huggingface.co/docs/diffusers/index) libraries and OpenVINO to accelerate end-to-end pipelines on Intel architectures. It provides ease-to-use cli interface for exporting models to [OpenVINO Intermediate Representation (IR)](https://docs.openvino.ai/2024/documentation/openvino-ir-format.html) format.

The command bellow demonstrates basic command for model export with `optimum-cli`

```
optimum-cli export openvino --model <model_id_or_path> --task <task> <out_dir>
```

where `--model` argument is model id from HuggingFace Hub or local directory with model (saved using `.save_pretrained` method), `--task ` is one of [supported task](https://huggingface.co/docs/optimum/exporters/task_manager) that exported model should solve. For LLMs it will be `text-generation-with-past`. If model initialization requires to use remote code, `--trust-remote-code` flag additionally should be passed.


### LLM conversion and Weights Compression using Optimum-CLI

[back to top ⬆️](#Table-of-contents:)

You can also apply fp16, 8-bit or 4-bit weight compression on the Linear, Convolutional and Embedding layers when exporting your model with the CLI by setting `--weight-format` to respectively fp16, int8 or int4. This type of optimization allows to reduce the memory footprint and inference latency.
By default the quantization scheme for int8/int4 will be [asymmetric](https://github.com/openvinotoolkit/nncf/blob/develop/docs/compression_algorithms/Quantization.md#asymmetric-quantization), to make it [symmetric](https://github.com/openvinotoolkit/nncf/blob/develop/docs/compression_algorithms/Quantization.md#symmetric-quantization) you can add `--sym`.

For INT4 quantization you can also specify the following arguments :

- The `--group-size` parameter will define the group size to use for quantization, -1 it will results in per-column quantization.
- The `--ratio` parameter controls the ratio between 4-bit and 8-bit quantization. If set to 0.9, it means that 90% of the layers will be quantized to int4 while 10% will be quantized to int8.

Smaller group_size and ratio values usually improve accuracy at the sacrifice of the model size and inference latency.

> **Note**: There may be no speedup for INT4/INT8 compressed models on dGPU.


In [8]:
from IPython.display import Markdown, display

prepare_int4_model = widgets.Checkbox(
    value=True,
    description="Prepare INT4 model",
    disabled=False,
)
prepare_int8_model = widgets.Checkbox(
    value=False,
    description="Prepare INT8 model",
    disabled=False,
)
prepare_fp16_model = widgets.Checkbox(
    value=False,
    description="Prepare FP16 model",
    disabled=False,
)

display(prepare_int4_model)
display(prepare_int8_model)
display(prepare_fp16_model)

Checkbox(value=True, description='Prepare INT4 model')

Checkbox(value=False, description='Prepare INT8 model')

Checkbox(value=False, description='Prepare FP16 model')

#### Weight compression with AWQ
[back to top ⬆️](#Table-of-contents:)

[Activation-aware Weight Quantization](https://arxiv.org/abs/2306.00978) (AWQ) is an algorithm that tunes model weights for more accurate INT4 compression. It slightly improves generation quality of compressed LLMs, but requires significant additional time for tuning weights on a calibration dataset. We use `wikitext-2-raw-v1/train` subset of the [Wikitext](https://huggingface.co/datasets/Salesforce/wikitext) dataset for calibration.

Below you can enable AWQ to be additionally applied during model export with INT4 precision.

>**Note**: Applying AWQ requires significant memory and time.

>**Note**: It is possible that there will be no matching patterns in the model to apply AWQ, in such case it will be skipped.

In [9]:
enable_awq = widgets.Checkbox(
    value=False,
    description="Enable AWQ",
    disabled=not prepare_int4_model.value,
)
display(enable_awq)

Checkbox(value=False, description='Enable AWQ')

In [10]:
from cmd_helper import optimum_cli

pt_model_id = llm_model_configuration["model_id"]
pt_model_name = llm_model_id.value.split("-")[0]
fp16_model_dir = Path(llm_model_id.value) / "FP16"
int8_model_dir = Path(llm_model_id.value) / "INT8_compressed_weights"
int4_model_dir = Path(llm_model_id.value) / "INT4_compressed_weights"


def convert_to_fp16():
    if (fp16_model_dir / "openvino_model.xml").exists():
        return
    remote_code = llm_model_configuration.get("remote_code", False)
    additional_args = {"task": "text-generation-with-past", "weight-format": "fp16"}
    if remote_code:
        additional_args["trust-remote-code"] = ""
    export_command_base = "optimum-cli export openvino --model {} --task text-generation-with-past --weight-format fp16".format(pt_model_id)
    if remote_code:
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " " + str(fp16_model_dir)
    display(Markdown("**Export command:**"))
    display(Markdown(f"`{export_command}`"))
    optimum_cli(pt_model_id, fp16_model_dir, show_command=False, additional_args=additional_args)


def convert_to_int8():
    if (int8_model_dir / "openvino_model.xml").exists():
        return
    int8_model_dir.mkdir(parents=True, exist_ok=True)
    remote_code = llm_model_configuration.get("remote_code", False)
    export_command_base = "optimum-cli export openvino --model {} --task text-generation-with-past --weight-format int8".format(pt_model_id)
    if remote_code:
        export_command_base += " --trust-remote-code"
    additional_args = {"task": "text-generation-with-past", "weight-format": "int8"}
    if remote_code:
        additional_args["trust-remote-code"] = ""
    export_command = export_command_base + " " + str(int8_model_dir)
    display(Markdown("**Export command:**"))
    display(Markdown(f"`{export_command}`"))
    optimum_cli(pt_model_id, int8_model_dir, show_command=False, additional_args=additional_args)


def convert_to_int4():
    compression_configs = {
        "zephyr-7b-beta": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "mistral-7b": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "minicpm-2b-dpo": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "gemma-2b-it": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "notus-7b-v1": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "neural-chat-7b-v3-1": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "llama-2-chat-7b": {
            "sym": True,
            "group_size": 128,
            "ratio": 0.8,
        },
        "llama-3-8b-instruct": {
            "sym": True,
            "group_size": 128,
            "ratio": 0.8,
        },
        "gemma-7b-it": {
            "sym": True,
            "group_size": 128,
            "ratio": 0.8,
        },
        "chatglm2-6b": {
            "sym": True,
            "group_size": 128,
            "ratio": 0.72,
        },
        "qwen-7b-chat": {"sym": True, "group_size": 128, "ratio": 0.6},
        "red-pajama-3b-chat": {
            "sym": False,
            "group_size": 128,
            "ratio": 0.5,
        },
        "qwen2.5-7b-instruct": {"sym": True, "group_size": 128, "ratio": 1.0},
        "qwen2.5-3b-instruct": {"sym": True, "group_size": 128, "ratio": 1.0},
        "qwen2.5-14b-instruct": {"sym": True, "group_size": 128, "ratio": 1.0},
        "qwen2.5-1.5b-instruct": {"sym": True, "group_size": 128, "ratio": 1.0},
        "qwen2.5-0.5b-instruct": {"sym": True, "group_size": 128, "ratio": 1.0},
        "default": {
            "sym": False,
            "group_size": 128,
            "ratio": 0.8,
        },
    }

    model_compression_params = compression_configs.get(llm_model_id.value, compression_configs["default"])
    if (int4_model_dir / "openvino_model.xml").exists():
        return
    remote_code = llm_model_configuration.get("remote_code", False)
    additional_args = {"task": "text-generation-with-past"}
    if remote_code:
        additional_args["trust-remote-code"] = ""
    export_command_base = "optimum-cli export openvino --model {} --task text-generation-with-past --weight-format int4".format(pt_model_id)
    int4_compression_args = " --group-size {} --ratio {}".format(model_compression_params["group_size"], model_compression_params["ratio"])
    additional_args.update({"weight-format": "int4", "group-size": model_compression_params["group_size"], "ratio": model_compression_params["ratio"]})
    if model_compression_params["sym"]:
        int4_compression_args += " --sym"
        additional_args["sym"] = ""
    if enable_awq.value:
        int4_compression_args += " --awq --dataset wikitext2 --num-samples 128"
        additional_args.update({"dataset": "wikitext2", "awq": "", "num-samples": "128"})
    export_command_base += int4_compression_args
    if remote_code:
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " " + str(int4_model_dir)
    display(Markdown("**Export command:**"))
    display(Markdown(f"`{export_command}`"))
    optimum_cli(pt_model_id, int4_model_dir, show_command=False, additional_args=additional_args)


if prepare_fp16_model.value:
    convert_to_fp16()
if prepare_int8_model.value:
    convert_to_int8()
if prepare_int4_model.value:
    convert_to_int4()

Let's compare model size for different compression types


In [11]:
fp16_weights = fp16_model_dir / "openvino_model.bin"
int8_weights = int8_model_dir / "openvino_model.bin"
int4_weights = int4_model_dir / "openvino_model.bin"

if fp16_weights.exists():
    print(f"Size of FP16 model is {fp16_weights.stat().st_size / 1024 / 1024:.2f} MB")
for precision, compressed_weights in zip([8, 4], [int8_weights, int4_weights]):
    if compressed_weights.exists():
        print(f"Size of model with INT{precision} compressed weights is {compressed_weights.stat().st_size / 1024 / 1024:.2f} MB")
    if compressed_weights.exists() and fp16_weights.exists():
        print(f"Compression rate for INT{precision} model: {fp16_weights.stat().st_size / compressed_weights.stat().st_size:.3f}")

Size of model with INT4 compressed weights is 1662.28 MB


### Convert embedding model using Optimum-CLI

[back to top ⬆️](#Table-of-contents:)

Since some embedding models can only support limited languages, we can filter them out according the LLM you selected.


In [12]:
embedding_model_id = list(SUPPORTED_EMBEDDING_MODELS[model_language.value])

embedding_model_id = widgets.Dropdown(
    options=embedding_model_id,
    value=embedding_model_id[0],
    description="Embedding Model:",
    disabled=False,
)

embedding_model_id

Dropdown(description='Embedding Model:', options=('bge-small-en-v1.5', 'bge-large-en-v1.5', 'bge-m3'), value='…

## Optional: Ingest Data from Telegram Channels

If you want to include Telegram channel messages in your RAG system, follow these steps:


In [15]:
# Import required modules for Telegram integration
import os
from dotenv import load_dotenv
from telegram_ingestion import TelegramChannelIngestion
from telegram_rag_integration import TelegramRAGIntegration
import asyncio

# Load environment variables for Telegram API credentials
load_dotenv()


True

In [23]:
# List your Telegram channels here (without @ symbol)
channels = ["guardian", "bloomberg"]

# Initialize the Telegram client and download messages
async def download_messages():
    ingestion = TelegramChannelIngestion(
        api_id=os.getenv("TELEGRAM_API_ID"),
        api_hash=os.getenv("TELEGRAM_API_HASH")
    )
    
    await ingestion.start()
    try:
        messages = await ingestion.process_channels(
            channels,
            limit_per_channel=100,  # Adjust as needed
            since_hours=24  # Adjust as needed
        )
        print(f"Downloaded {len(messages)} messages from {len(channels)} channels")
    finally:
        await ingestion.stop()

# Run the download
await download_messages()


Downloaded 200 messages from 2 channels


In [24]:
# Initialize the RAG integration with OpenVINO-optimized models
telegram_rag = TelegramRAGIntegration(
    embedding_model_name="BAAI/bge-small-en-v1.5",  # Use the same model as the main RAG system
    vector_store_path="telegram_vector_store",
    chunk_size=500,
    chunk_overlap=50,
    device="CPU"  # Use the same device as the main RAG system
)

# Process all downloaded messages
telegram_rag.process_telegram_data_dir()


TypeError: Core.compile_model() got an unexpected keyword argument 'device'

In [ ]:
def query_telegram_messages(query: str, k: int = 5, channel: str = None):
    """
    Query Telegram messages with optional channel filtering
    
    Args:
        query: The search query
        k: Number of results to return
        channel: Optional channel name to filter results
    """
    filter_dict = {"channel": channel} if channel else None
    results = telegram_rag.query_messages(query, k=k, filter_dict=filter_dict)
    
    print(f"Query: {query}\n")
    for i, doc in enumerate(results, 1):
        print(f"Result {i}:")
        print(f"Channel: {doc.metadata['channel']}")
        print(f"Date: {doc.metadata['date']}")
        print(f"Content: {doc.page_content[:200]}...\n")

# Example queries
print("Querying all channels:")
query_telegram_messages("What are the latest announcements?")

print("\nQuerying a specific channel:")
query_telegram_messages("What are the latest updates?", channel="example_channel1")


In [ ]:
def query_all_sources(query: str, k_per_source: int = 3):
    """
    Query both document and Telegram sources
    
    Args:
        query: The search query
        k_per_source: Number of results to return from each source
    """
    print(f"Query: {query}\n")
    
    # Get results from documents
    print("Results from documents:")
    doc_results = qa_chain.invoke({"question": query})
    print(doc_results["answer"])
    print("\n" + "-"*50 + "\n")
    
    # Get results from Telegram
    print("Results from Telegram channels:")
    telegram_results = telegram_rag.query_messages(query, k=k_per_source)
    for i, doc in enumerate(telegram_results, 1):
        print(f"Result {i}:")
        print(f"Channel: {doc.metadata['channel']}")
        print(f"Date: {doc.metadata['date']}")
        print(f"Content: {doc.page_content[:200]}...\n")

# Example combined query
query_all_sources("What are the latest updates and announcements?")


In [13]:
embedding_model_configuration = SUPPORTED_EMBEDDING_MODELS[model_language.value][embedding_model_id.value]
print(f"Selected {embedding_model_id.value} model")

Selected bge-small-en-v1.5 model


OpenVINO embedding model and tokenizer can be exported by `feature-extraction` task with `optimum-cli`.


In [14]:
export_command_base = "optimum-cli export openvino --model {} --task feature-extraction".format(embedding_model_configuration["model_id"])
export_command = export_command_base + " " + str(embedding_model_id.value)

display(Markdown("**Export command:**"))
display(Markdown(f"`{export_command}`"))

if not Path(embedding_model_id.value).exists():
    optimum_cli(embedding_model_configuration["model_id"], str(embedding_model_id.value), show_command=False, additional_args={"task": "feature-extraction"})

**Export command:**

`optimum-cli export openvino --model BAAI/bge-small-en-v1.5 --task feature-extraction bge-small-en-v1.5`

### Convert rerank model using Optimum-CLI

[back to top ⬆️](#Table-of-contents:)


In [15]:
rerank_model_id = list(SUPPORTED_RERANK_MODELS)

rerank_model_id = widgets.Dropdown(
    options=rerank_model_id,
    value=rerank_model_id[0],
    description="Rerank Model:",
    disabled=False,
)

rerank_model_id

Dropdown(description='Rerank Model:', options=('bge-reranker-v2-m3', 'bge-reranker-large', 'bge-reranker-base'…

In [16]:
rerank_model_configuration = SUPPORTED_RERANK_MODELS[rerank_model_id.value]
print(f"Selected {rerank_model_id.value} model")

Selected bge-reranker-v2-m3 model


Since `rerank` model is sort of sentence classification task, its OpenVINO IR and tokenizer can be exported by `text-classification` task with `optimum-cli`.


In [17]:
export_command_base = "optimum-cli export openvino --model {} --task text-classification".format(rerank_model_configuration["model_id"])
export_command = export_command_base + " " + str(rerank_model_id.value)

display(Markdown("**Export command:**"))
display(Markdown(f"`{export_command}`"))

if not Path(rerank_model_id.value).exists():
    optimum_cli(rerank_model_configuration["model_id"], str(rerank_model_id.value), show_command=False, additional_args={"task": "text-classification"})

**Export command:**

`optimum-cli export openvino --model BAAI/bge-reranker-v2-m3 --task text-classification bge-reranker-v2-m3`

## Select device for inference and model variant

[back to top ⬆️](#Table-of-contents:)

> **Note**: There may be no speedup for INT4/INT8 compressed models on dGPU.

### Select device for embedding model inference

[back to top ⬆️](#Table-of-contents:)


In [18]:
from notebook_utils import device_widget

embedding_device = device_widget()

embedding_device

Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'AUTO'), value='AUTO')

In [20]:
print(f"Embedding model will be loaded to {embedding_device.value} device for text embedding")

Embedding model will be loaded to AUTO device for text embedding


Optimize the BGE embedding model's parameter precision when loading model to NPU device.

In [21]:
from notebook_utils import optimize_bge_embedding

USING_NPU = embedding_device.value == "NPU"

npu_embedding_dir = embedding_model_id.value + "-npu"
npu_embedding_path = Path(npu_embedding_dir) / "openvino_model.xml"
if USING_NPU and not Path(npu_embedding_dir).exists():
    shutil.copytree(embedding_model_id.value, npu_embedding_dir)
    optimize_bge_embedding(Path(embedding_model_id.value) / "openvino_model.xml", npu_embedding_path)

### Select device for rerank model inference

[back to top ⬆️](#Table-of-contents:)


In [22]:
rerank_device = device_widget()

rerank_device

Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'AUTO'), value='AUTO')

In [23]:
print(f"Rerenk model will be loaded to {rerank_device.value} device for text reranking")

Rerenk model will be loaded to AUTO device for text reranking


### Select device for LLM model inference

[back to top ⬆️](#Table-of-contents:)


In [24]:
from notebook_utils import device_widget

llm_device = device_widget()

llm_device

Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'AUTO'), value='AUTO')

In [25]:
print(f"LLM model will be loaded to {llm_device.value} device for response generation")

LLM model will be loaded to AUTO device for response generation


## Load models

[back to top ⬆️](#Table-of-contents:)

### Load embedding model

[back to top ⬆️](#Table-of-contents:)

Now a local embedding model of OpenVINO can be accelerated on NPU by using the `OpenVINOBgeEmbeddings` or `OpenVINOEmbeddings` class without PyTorch requirements.


In [26]:
from ov_langchain_helper import OpenVINOBgeEmbeddings

embedding_model_name = npu_embedding_dir if USING_NPU else embedding_model_id.value
batch_size = 1 if USING_NPU else 4
embedding_model_kwargs = {"device_name": embedding_device.value}
encode_kwargs = {
    "mean_pooling": embedding_model_configuration["mean_pooling"],
    "normalize_embeddings": embedding_model_configuration["normalize_embeddings"],
    "batch_size": batch_size,
}

if USING_NPU:
    import openvino as ov

    core = ov.Core()
    embedding_model = core.read_model(Path(embedding_model_name) / "openvino_model.xml")
    port_to_shape = dict()
    for input_port in embedding_model.inputs:
        port_to_shape[input_port] = [1, 512]
    embedding_model.reshape(port_to_shape)
    embedding_model = core.compile_model(embedding_model, embedding_device.value)
    embedding = OpenVINOBgeEmbeddings(
        ov_model=embedding_model,
        model_path=embedding_model_name,
        model_kwargs=embedding_model_kwargs,
        encode_kwargs=encode_kwargs,
    )
else:
    embedding = OpenVINOBgeEmbeddings(
        model_path=embedding_model_name,
        model_kwargs=embedding_model_kwargs,
        encode_kwargs=encode_kwargs,
    )

text = "This is a test document."
embedding_result = embedding.embed_query(text)
embedding_result[:3]

array([-0.04208659,  0.06681865,  0.00791674], dtype=float32)

In [27]:
print(len(embedding_result))

384


### Load rerank model

[back to top ⬆️](#Table-of-contents:)

Now a local rerank model of OpenVINO can be accelerated on NPU by using the `OpenVINOReranker` class without PyTorch requirements.

> **Note**: Rerank can be skipped in RAG.


In [28]:
from ov_langchain_helper import OpenVINOReranker

rerank_model_name = rerank_model_id.value
rerank_model_kwargs = {"device_name": rerank_device.value}
rerank_top_n = 2
vector_search_top_k_npu = 4

if rerank_device.value == "NPU":
    import openvino as ov

    core = ov.Core()
    rerank_model = core.read_model(Path(rerank_model_name) / "openvino_model.xml")
    port_to_shape = dict()
    for input_port in rerank_model.inputs:
        port_to_shape[input_port] = [vector_search_top_k_npu, 512]
    rerank_model.reshape(port_to_shape)

    rerank_model = core.compile_model(rerank_model, rerank_device.value)

    reranker = OpenVINOReranker(
        ov_model=rerank_model,
        model_path=rerank_model_name,
        top_n=rerank_top_n,
    )
else:
    reranker = OpenVINOReranker(
        model_path=rerank_model_name,
        model_kwargs=rerank_model_kwargs,
        top_n=rerank_top_n,
    )

### Load LLM model

[back to top ⬆️](#Table-of-contents:)

`OpenVINOLLM` is a llm wrapper of [openvino_genai API](https://github.com/openvinotoolkit/openvino.genai) for LangChain. Models can be loaded by specifying the model parameters using the `from_model_path` method. If you have an Intel GPU or NPU, you can specify `deivce="GPU"` or `deivce="NPU"` to run inference on it.

In [29]:
available_models = []
if int4_model_dir.exists():
    available_models.append("INT4")
if int8_model_dir.exists():
    available_models.append("INT8")
if fp16_model_dir.exists():
    available_models.append("FP16")

model_to_run = widgets.Dropdown(
    options=available_models,
    value=available_models[0],
    description="Model to run:",
    disabled=False,
)

model_to_run

Dropdown(description='Model to run:', options=('INT4',), value='INT4')

`OpenVINOLLM` can support customized tokenizer for better accuracy with some specific models. For example, the default OpenVINO tokenizer can be replaced with a Transformer library based tokenizer through following steps:

```Python
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(str(model_dir))
llm = OpenVINOLLM.from_model_path(
    model_path=str(model_dir),
    device=llm_device.value,
    tokenizer=tokenizer,
)
```

In [30]:
from ov_langchain_helper import OpenVINOLLM


if model_to_run.value == "INT4":
    model_dir = int4_model_dir
elif model_to_run.value == "INT8":
    model_dir = int8_model_dir
else:
    model_dir = fp16_model_dir
print(f"Loading model from {model_dir}")


llm = OpenVINOLLM.from_model_path(
    model_path=str(model_dir),
    device=llm_device.value,
    # tokenizer=tokenizer,
)

llm.config.max_new_tokens = 2
llm.invoke("2 + 2 =")

Loading model from qwen2.5-3b-instruct/INT4_compressed_weights


'2 +'

## Run QA over Document

[back to top ⬆️](#Table-of-contents:)

Now, when model created, we can setup Chatbot interface using [Gradio](https://www.gradio.app/).

A typical RAG application has two main components:

- **Indexing**: a pipeline for ingesting data from a source and indexing it. This usually happen offline.

- **Retrieval and generation**: the actual RAG chain, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

The most common full sequence from raw data to answer looks like:

**Indexing**

1. `Load`: First we need to load our data. We’ll use DocumentLoaders for this.
2. `Split`: Text splitters break large Documents into smaller chunks. This is useful both for indexing data and for passing it in to a model, since large chunks are harder to search over and won’t in a model’s finite context window.
3. `Store`: We need somewhere to store and index our splits, so that they can later be searched over. This is often done using a VectorStore and Embeddings model.

![Indexing pipeline](https://github.com/openvinotoolkit/openvino_notebooks/assets/91237924/dfed2ba3-0c3a-4e0e-a2a7-01638730486a)

**Retrieval and generation**

1. `Retrieve`: Given a user input, relevant splits are retrieved from storage using a Retriever.
2. `Generate`: A LLM produces an answer using a prompt that includes the question and the retrieved data.

![Retrieval and generation pipeline](https://github.com/openvinotoolkit/openvino_notebooks/assets/91237924/f0545ddc-c0cd-4569-8c86-9879fdab105a)


In [31]:
import re
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    MarkdownTextSplitter,
)
from langchain.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    PyPDFLoader,
    TextLoader,
    UnstructuredEPubLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredODTLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
)


class ChineseTextSplitter(CharacterTextSplitter):
    def __init__(self, pdf: bool = False, **kwargs):
        super().__init__(**kwargs)
        self.pdf = pdf

    def split_text(self, text: str) -> list[str]:
        if self.pdf:
            text = re.sub(r"\n{3,}", "\n", text)
            text = text.replace("\n\n", "")
        sent_sep_pattern = re.compile('([﹒﹔﹖﹗．。！？]["’”」』]{0,2}|(?=["‘“「『]{1,2}|$))')
        sent_list = []
        for ele in sent_sep_pattern.split(text):
            if sent_sep_pattern.match(ele) and sent_list:
                sent_list[-1] += ele
            elif ele:
                sent_list.append(ele)
        return sent_list


TEXT_SPLITERS = {
    "Character": CharacterTextSplitter,
    "RecursiveCharacter": RecursiveCharacterTextSplitter,
    "Markdown": MarkdownTextSplitter,
    "Chinese": ChineseTextSplitter,
}


LOADERS = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".enex": (EverNoteLoader, {}),
    ".epub": (UnstructuredEPubLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".odt": (UnstructuredODTLoader, {}),
    ".pdf": (PyPDFLoader, {}),
    ".ppt": (UnstructuredPowerPointLoader, {}),
    ".pptx": (UnstructuredPowerPointLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"}),
}

if model_language.value == "English":
    text_example_path = "text_example_en.pdf"
else:
    text_example_path = "text_example_cn.pdf"

We can build a RAG pipeline of LangChain through [`create_retrieval_chain`](https://python.langchain.com/docs/modules/chains/), which will help to create a chain to connect RAG components including:

- [`Vector stores`](https://python.langchain.com/docs/modules/data_connection/vectorstores/)，
- [`Retrievers`](https://python.langchain.com/docs/modules/data_connection/retrievers/)
- [`LLM`](https://python.langchain.com/docs/integrations/llms/)
- [`Embedding`](https://python.langchain.com/docs/integrations/text_embedding/)


In [32]:
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.docstore.document import Document
from langchain.retrievers import ContextualCompressionRetriever
import gradio as gr

rag_prompt_template = llm_model_configuration["rag_prompt_template"]


def load_single_document(file_path: str) -> list[Document]:
    """
    helper for loading a single document

    Params:
      file_path: document path
    Returns:
      documents loaded

    """
    ext = "." + file_path.rsplit(".", 1)[-1]
    if ext in LOADERS:
        loader_class, loader_args = LOADERS[ext]
        loader = loader_class(file_path, **loader_args)
        return loader.load()

    raise ValueError(f"File does not exist '{ext}'")


def default_partial_text_processor(partial_text: str, new_text: str):
    """
    helper for updating partially generated answer, used by default

    Params:
      partial_text: text buffer for storing previosly generated text
      new_text: text update for the current step
    Returns:
      updated text string

    """
    partial_text += new_text
    return partial_text


text_processor = llm_model_configuration.get("partial_text_processor", default_partial_text_processor)


def create_vectordb(
    docs, spliter_name, chunk_size, chunk_overlap, vector_search_top_k, vector_rerank_top_n, run_rerank, search_method, score_threshold, progress=gr.Progress()
):
    """
    Initialize a vector database

    Params:
      doc: orignal documents provided by user
      spliter_name: spliter method
      chunk_size:  size of a single sentence chunk
      chunk_overlap: overlap size between 2 chunks
      vector_search_top_k: Vector search top k
      vector_rerank_top_n: Search rerank top n
      run_rerank: whether run reranker
      search_method: top k search method
      score_threshold: score threshold when selecting 'similarity_score_threshold' method

    """
    global db
    global retriever
    global combine_docs_chain
    global rag_chain

    if rerank_device.value == "NPU":
        vector_search_top_k = vector_search_top_k_npu
    if vector_rerank_top_n > vector_search_top_k:
        gr.Warning("Search top k must >= Rerank top n")

    documents = []
    for doc in docs:
        if type(doc) is not str:
            doc = doc.name
        documents.extend(load_single_document(doc))

    text_splitter = TEXT_SPLITERS[spliter_name](chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    texts = text_splitter.split_documents(documents)
    db = FAISS.from_documents(texts, embedding)
    if search_method == "similarity_score_threshold":
        search_kwargs = {"k": vector_search_top_k, "score_threshold": score_threshold}
    else:
        search_kwargs = {"k": vector_search_top_k}
    retriever = db.as_retriever(search_kwargs=search_kwargs, search_type=search_method)
    if run_rerank:
        reranker.top_n = vector_rerank_top_n
        retriever = ContextualCompressionRetriever(base_compressor=reranker, base_retriever=retriever)
    prompt = PromptTemplate.from_template(rag_prompt_template)
    combine_docs_chain = create_stuff_documents_chain(llm, prompt)

    rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

    return "Vector database is Ready"


def update_retriever(vector_search_top_k, vector_rerank_top_n, run_rerank, search_method, score_threshold):
    """
    Update retriever

    Params:
      vector_search_top_k: Vector search top k
      vector_rerank_top_n: Search rerank top n
      run_rerank: whether run reranker
      search_method: top k search method
      score_threshold: score threshold when selecting 'similarity_score_threshold' method

    """
    global retriever
    global rag_chain

    if rerank_device.value == "NPU":
        vector_search_top_k = vector_search_top_k_npu
    if vector_rerank_top_n > vector_search_top_k:
        gr.Warning("Search top k must >= Rerank top n")

    if search_method == "similarity_score_threshold":
        search_kwargs = {"k": vector_search_top_k, "score_threshold": score_threshold}
    else:
        search_kwargs = {"k": vector_search_top_k}
    retriever = db.as_retriever(search_kwargs=search_kwargs, search_type=search_method)
    if run_rerank:
        retriever = ContextualCompressionRetriever(base_compressor=reranker, base_retriever=retriever)
        reranker.top_n = vector_rerank_top_n
    rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

    return "Vector database is Ready"


# initialize the vector store with example document
create_vectordb(
    [text_example_path],
    "RecursiveCharacter",
    chunk_size=400,
    chunk_overlap=50,
    vector_search_top_k=4,
    vector_rerank_top_n=2,
    run_rerank=True,
    search_method="similarity" if rerank_device.value == "NPU" else "similarity_score_threshold",
    score_threshold=0.5,
)


def bot(history, temperature, top_p, top_k, repetition_penalty, hide_full_prompt, do_rag):
    """
    callback function for running chatbot on submit button click

    Params:
      history: conversation history
      temperature:  parameter for control the level of creativity in AI-generated text.
                    By adjusting the `temperature`, you can influence the AI model's probability distribution, making the text more focused or diverse.
      top_p: parameter for control the range of tokens considered by the AI model based on their cumulative probability.
      top_k: parameter for control the range of tokens considered by the AI model based on their cumulative probability, selecting number of tokens with highest probability.
      repetition_penalty: parameter for penalizing tokens based on how frequently they occur in the text.
      hide_full_prompt: whether to show searching results in promopt.
      do_rag: whether do RAG when generating texts.
    """

    llm.config.temperature = temperature
    llm.config.top_p = 1
    llm.config.top_k = 2
    llm.config.do_sample = temperature > 0.0
    llm.config.max_new_tokens = 1024
    llm.config.repetition_penalty = repetition_penalty

    partial_text = ""
    if do_rag:
        streaming_response = rag_chain.stream({"input": history[-1][0]})
        for new_text in streaming_response:
            if not hide_full_prompt and list(new_text.keys())[0] != "answer":
                partial_text = text_processor(partial_text, str(new_text))
                history[-1][1] = partial_text
                yield history
            if list(new_text.keys())[0] == "answer":
                partial_text = text_processor(partial_text, list(new_text.values())[0])
                history[-1][1] = partial_text
                yield history
    else:
        streaming_response = llm.stream(history[-1][0])
        for new_text in streaming_response:
            partial_text = text_processor(partial_text, new_text)
            history[-1][1] = partial_text
            yield history

Next we can create a Gradio UI and run demo.

In [33]:
streaming_response = rag_chain.stream({"input": "hi"})
for new_text in streaming_response:
    if list(new_text.keys())[0] == "answer":
        print(list(new_text.values())[0], end="", flush=True)

No relevant docs were retrieved using the relevance score threshold 0.5


Hello!

In [ ]:
if not Path("gradio_helper.py").exists():
    r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/notebooks/llm-rag-langchain/gradio_helper.py")
    open("gradio_helper.py", "w").write(r.text)

from gradio_helper import make_demo

demo = make_demo(
    load_doc_fn=create_vectordb,
    run_fn=bot,
    stop_fn=None,
    update_retriever_fn=update_retriever,
    model_name=llm_model_id.value,
    language=model_language.value,
)

try:
    demo.queue().launch(debug=True)
except Exception:
    demo.queue().launch(share=True, debug=True)
# If you are launching remotely, specify server_name and server_port
# EXAMPLE: `demo.launch(server_name='your server name', server_port='server port in int')`
# To learn more please refer to the Gradio docs: https://gradio.app/docs/

/home/sidd/Documents/GitHub/openvino_notebooks/notebooks/llm-rag-langchain/gradio_helper.py:170: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
